In [1]:
import numpy as np
import json
from tournament import Tournament


with open('teams.json', 'r') as f:
    teams = json.load(f)

with open('locations.json', 'r') as f:
    locations = json.load(f)

stadiums = {}
for loc in locations:
    snames = []
    for s in locations[loc]['stadiums']:
        snames.append(s)
        stadiums[s] = {
            'location': loc,
            'size': locations[loc]['stadiums'][s]['size'] 
        }
    
    locations[loc]['stadiums'] = snames

timeslot_values = [100,130,40,50,80,110,50,40,90] # Change later according to attendances
timeslot_names = ['Thursday Night','Friday Night','Saturday Morning','Saturday Afternoon','Saturday Evening',
                  'Saturday Night','Sunday Afternoon','Sunday Evening', 'Sunday Night']
timeslots = [{'value': v, 'name': n} for (n,v) in zip(timeslot_names, timeslot_values)]
tourn = Tournament(teams = teams, locations = locations, stadiums=stadiums, timeslots = timeslots, rounds = 22)


In [2]:
tourn.fixture_matrix.shape

(18, 18, 9, 9, 22)

In [3]:
def find_best_inround(attractiveness_matrix, round = None):
    if round is not None:
        x = attractiveness_matrix[:,:,:,:,round]
    else:
        x = attractiveness_matrix
    return tuple(np.unravel_index(np.argmax(x), x.shape))

def find_k_best_inround(attractiveness_matrix, k, round = None, ):
    if round is not None:
        x = attractiveness_matrix[:,:,:,:,round]
    else:
        x = attractiveness_matrix
    flat = x.flatten()

    # Find the indices of the N largest values in the flattened array
    indices = np.argpartition(flat, -k)[-k:]
    # Convert the flattened indices back to the indices in the original matrix
    return [tuple(np.unravel_index(idx, x.shape)) for idx in indices]

# deletes a pair of teams (order matters)
def delete_team_pair(matrix, i, j):
    # todo, might not be worth it compared to setting attractiveness to -1


    pass

def adapt_greedy(attractiveness_matrix, tmp_matrix, round, game_chosen):
    team1, team2, stadium, timeslot = game_chosen
    tmp_matrix[:,:,:,timeslot] = -1
    tmp_matrix[team1, team2, :, :] = -1

    # check if a stadium has already been used two times in a day
    if timeslot in [2,3,4]:
        tmp_matrix[:,:,stadium,2] = -1
        tmp_matrix[:,:,stadium,3] = -1
        tmp_matrix[:,:,stadium,4] = -1
    if timeslot in [5,6]:
        tmp_matrix[:,:,stadium,5] = -1
        tmp_matrix[:,:,stadium,6] = -1 
    pass

def maintain_feasibility(tmp_matrix, game_chosen):
    team1, team2, stadium, timeslot = game_chosen
    # delete the timeslot and games played between the two teams for future considerations
    # this is done by setting the attractiveness to -1
    tmp_matrix[:,:,:,timeslot] = -1
    tmp_matrix[team1, team2, :, :] = -1

    # check if a stadium has already been used two times in a day
    if timeslot in [2,3,4]:
        tmp_matrix[:,:,stadium,2] = -1
        tmp_matrix[:,:,stadium,3] = -1
        tmp_matrix[:,:,stadium,4] = -1
    if timeslot in [5,6]:
        tmp_matrix[:,:,stadium,5] = -1
        tmp_matrix[:,:,stadium,6] = -1 
    
    return tmp_matrix

def construct_greedy_round(attractiveness_matrix, round, timeslots, track_games = False,
                            fixture_matrix = None):
    games = [None for _ in range(timeslots)]
    games_fulfilled = 0
    tmp_matrix = attractiveness_matrix[:,:,:,:,round]

    # matrix to return to keep track of played games between two teams
    if track_games: return_matrix = attractiveness_matrix


    while games_fulfilled < timeslots:
        new_game = find_best_inround(tmp_matrix)
        tmp_matrix = maintain_feasibility(tmp_matrix=tmp_matrix, game_chosen=new_game)
        games_fulfilled += 1
        
        team1, team2, stadium, timeslot = new_game
        tourn.print_game(team1, team2, stadium, round, timeslot)
        games[timeslot] = [team1,team2,stadium]
        # if we are running this in a loop to construct the whole schedule we want to keep track of the games played
        # between teams across all rounds
        if track_games: return_matrix[team1,team2,:,:,:] = -1
        if fixture_matrix is not None: fixture_matrix[team1, team2, stadium, timeslot, round] = 1
    
    if track_games:
        if fixture_matrix is not None:
            return games, return_matrix, fixture_matrix
        return games, return_matrix
    if fixture_matrix is not None:
        return games, fixture_matrix
    return games

def construct_greedy_schedule(attractiveness_matrix, rounds, timeslots):
    tmp_matrix = attractiveness_matrix.copy()
    fixture = np.zeros(shape = attractiveness_matrix.shape)
    schedule = [None for _ in range(rounds)]
    for round in range(rounds):
        print(f"Round {round}: ")
        games, tmp_matrix, fixture = construct_greedy_round(tmp_matrix, round, timeslots, 
                                                            track_games=True,
                                                            fixture_matrix=fixture)
        schedule[round] = games
    
    return schedule, fixture


def construct_greedy_round_random(attractiveness_matrix, round, timeslots,
                                    rcl_length = 10,
                                    track_games = False,
                                    fixture_matrix = None):
    games = [None for _ in range(timeslots)]
    games_fulfilled = 0
    tmp_matrix = attractiveness_matrix[:,:,:,:,round]

    # matrix to return to keep track of played games between two teams
    if track_games: return_matrix = attractiveness_matrix
    while games_fulfilled < timeslots:
        candidate_list = find_k_best_inround(tmp_matrix, k = rcl_length)

        # select at random
        new_game = candidate_list[np.random.choice(rcl_length)]
        tmp_matrix = maintain_feasibility(tmp_matrix=tmp_matrix, game_chosen=new_game)
        
        games_fulfilled += 1
        
        team1, team2, stadium, timeslot = new_game
        tourn.print_game(team1, team2, stadium, round, timeslot)
        games[timeslot] = [team1,team2,stadium]

        # if we are running this in a loop to construct the whole schedule we want to keep track of the games played
        # between teams across all rounds
        if track_games: return_matrix[team1,team2,:,:,:] = -1
        if fixture_matrix is not None: fixture_matrix[team1, team2, stadium, timeslot, round] = 1
    
    if track_games:
        if fixture_matrix is not None:
            return games, return_matrix, fixture_matrix
        return games, return_matrix
    if fixture_matrix is not None:
        return games, fixture_matrix
    return games
    

def construct_greedy_schedule_random(attractiveness_matrix, rounds, timeslots, rcl_length = 10):
    tmp_matrix = attractiveness_matrix.copy()
    fixture = np.zeros(shape = attractiveness_matrix.shape)
    schedule = [None for _ in range(rounds)]
    for round in range(rounds):
        print(f"Round {round}: ")
        games, tmp_matrix, fixture = construct_greedy_round_random(tmp_matrix, round, timeslots, 
                                                                    rcl_length=rcl_length,
                                                                    track_games=True,
                                                                    fixture_matrix=fixture)
        schedule[round] = games
    
    return schedule, fixture



In [5]:
arr = tourn.weight_matrix
result, fixture = construct_greedy_schedule(arr, rounds = 22, timeslots = 9)

Round 0: 
Friday Night: Collingwood Magpies vs. Brisbane Lions at MCG
Saturday Night: Collingwood Magpies vs. Melbourne Demons at MCG
Thursday Night: Melbourne Demons vs. Collingwood Magpies at MCG
Sunday Night: Collingwood Magpies vs. Port Adelaide Power at MCG
Saturday Evening: Brisbane Lions vs. Collingwood Magpies at Gabba
Saturday Afternoon: Carlton Blues vs. Melbourne Demons at MCG
Sunday Evening: Melbourne Demons vs. Carlton Blues at MCG
Sunday Afternoon: Port Adelaide Power vs. Collingwood Magpies at Adelaide Oval
Saturday Morning: Carlton Blues vs. Collingwood Magpies at Marvel
Round 1: 
Friday Night: Collingwood Magpies vs. Carlton Blues at MCG
Saturday Night: Melbourne Demons vs. Port Adelaide Power at MCG
Thursday Night: Essendon Bombers vs. Richmond Tigers at MCG
Sunday Night: Richmond Tigers vs. Essendon Bombers at MCG
Saturday Evening: Carlton Blues vs. St Kilda Saints at Marvel
Sunday Afternoon: St Kilda Saints vs. Carlton Blues at Marvel
Saturday Afternoon: Port Adelai

In [6]:
fixture.shape

(18, 18, 9, 9, 22)

In [7]:
tourn.fixture_attractiveness(fixture=fixture)

1916994.0161365855

In [8]:
tourn.feasibility(fixture=fixture, debug=True)

Violated constraint for total number of home games for team Adelaide Crows
Violated constraint - Adelaide Crows did not play Brisbane Lions
Violated constraint - Adelaide Crows did not play Gold Coast Suns
Violated constraint - Adelaide Crows did not play Hawthorn Hawks
Violated constraint - Adelaide Crows did not play North Melbourne Kangaroos
Violated constraint - Adelaide Crows did not play West Coast Eagles
Violated constraint for total number of home games for team Brisbane Lions
Violated constraint - Brisbane Lions did not play Adelaide Crows
Violated constraint - Brisbane Lions did not play Essendon Bombers
Violated constraint - Brisbane Lions did not play Fremantle Dockers
Violated constraint - Brisbane Lions did not play Geelong Cats
Violated constraint - Brisbane Lions did not play Gold Coast Suns
Violated constraint - Brisbane Lions did not play Hawthorn Hawks
Violated constraint - Brisbane Lions did not play North Melbourne Kangaroos
Violated constraint - Brisbane Lions did

(126.0, 0.0)

In [8]:
arr = tourn.attractiveness_matrix
result, fixture = construct_greedy_schedule_random(arr, rounds = 22, timeslots = 7)

Round 0: 
Friday Night: Brisbane Lions vs. Collingwood Magpies at MCG
Sunday Afternoon: Collingwood Magpies vs. Collingwood Magpies at MCG
Saturday Night: Collingwood Magpies vs. Brisbane Lions at Adelaide Oval
Thursday Night: Port Adelaide Power vs. Collingwood Magpies at Marvel
Saturday Evening: Collingwood Magpies vs. Port Adelaide Power at MCG
Sunday Evening: Brisbane Lions vs. Port Adelaide Power at Adelaide Oval
Saturday Afternoon: Brisbane Lions vs. Brisbane Lions at Gabba
Round 1: 
Saturday Night: Collingwood Magpies vs. Melbourne Demons at MCG
Friday Night: Carlton Blues vs. Carlton Blues at MCG
Thursday Night: Melbourne Demons vs. Melbourne Demons at MCG
Sunday Afternoon: Collingwood Magpies vs. Carlton Blues at MCG
Saturday Afternoon: Melbourne Demons vs. Port Adelaide Power at Adelaide Oval
Saturday Evening: Geelong Cats vs. Essendon Bombers at Marvel
Sunday Evening: Port Adelaide Power vs. Brisbane Lions at Optus
Round 2: 
Saturday Night: Melbourne Demons vs. Collingwood M

In [9]:
tourn.fixture_attractiveness(fixture=fixture)

1540000.0

In [10]:
tourn.feasibility(fixture=fixture)

(333.0, 18.0)